In [ ]:
import argparse
import requests
import subprocess
from time import sleep
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

First, put your token and folder id.

Follow Yandex.Cloud guide for more information about the SpeechKit usage.

In [ ]:
token = '<YOUR TOKEN>'
folder_id = '<FOLDER ID>'

Speech synthesize method

In [ ]:
def synthesize(folder_id, iam_token, text):
    url = 'https://tts.api.cloud.yandex.net/speech/v1/tts:synthesize'
    headers = {
        'Authorization': 'Bearer ' + iam_token,
    }

    data = {
        'text': text,
        'lang': 'ru-RU',
        'voice': np.random.choice(['alena', 'filipp']),
        'folderId': folder_id,
        'emotion': np.random.choice(['evil', 'neutral', 'good']),
        'speed' : 1.1,
        'format': 'lpcm',
        'sampleRateHertz': 48000,
    }

    with requests.post(url, headers=headers, data=data, stream=True) as resp:
        if resp.status_code != 200:
            raise RuntimeError("Invalid response received: code: %d, message: %s" % (resp.status_code, resp.text))

        for chunk in resp.iter_content(chunk_size=None):
            yield chunk

Define a file with your source texts separated by `\n`

In [ ]:
source_texts = 'source_texts.txt'

In [ ]:
with open('source_texts.txt', 'r') as handler:
    sentences = [x.strip() for x in handler.readlines()]

Let's start the generating process.

In [ ]:
gt = []

for idx, sntnc in tqdm(enumerate(sentences), total=len(sentences)):

    output_tmp = 'tmp_recordings/' + str(idx) + '.raw'
    output_fin = 'final_recordings/' + str(idx) + '.wav'
    
    gt.append(f"{str(idx)}.wav\t{sntnc}")

    bashCommand = "./sox-14.4.2/sox -r 48000 -b 16 -e signed-integer -c 1 " + output_tmp + " " + output_fin


    with open(output_tmp, "wb") as f:
        for audio_content in synthesize(folder_id, token, sntnc):
            f.write(audio_content)

    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    sleep(0.02)
with open('final_recordings/gt.txt', 'w') as f:
    f.write('\n'.join(gt))

Now you can upload generated files to Yandex.Cloud or any other data storage with direct access.
(It can also be automated, but we do not provide a code for this process since it strongly depends on where the data will be stored).

In [ ]:
base_url = '<put your base url like https://storage.yandexcloud.net/<folder>'

In [ ]:
with open('result.tsv', 'w') as f:
    f.write('INPUT:audio\n')
    lines = []
    for i in range(len(gt)):
        lines.append(f'{base_url}/{i}.wav')
    f.write('\n'.join(lines))